<a href="https://colab.research.google.com/github/keshavvprabhu/FastAPITutorials/blob/master/SignatureVerification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: Write a program that takes two signature images, preprocesses them and then compares the original and identifies the fake or forged signature and gives a confidence percentage. The below code gives me > 100% confidence which is incorrect. This confidence percentage is still a little too much on the higher side. It seems to be overfitting. Can you tweak the confidence percentage to make it better
# the match percentage is skewed and does not represent correctness.
# Use other methods or models to verify the same thing
# After the latest changes, things are looking better.
# Create multiple compare_signatures() with different skimage, open cv models
# please fix the error

import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def compare_signatures_ssim(image1_path, image2_path):
    # Load images in grayscale
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

    # Resize images to the same dimensions (important for comparison)
    img1 = cv2.resize(img1, (256, 256))
    img2 = cv2.resize(img2, (256, 256))

    # Calculate SSIM
    score, diff = ssim(img1, img2, full=True)

    # Confidence percentage (adjusted scale)
    confidence = (score * 100)

    return confidence

def compare_signatures_orb(image1_path, image2_path):
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

    # Initiate ORB detector
    orb = cv2.ORB_create()

    # find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)

    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors.
    matches = bf.match(des1,des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)

    # Calculate the number of good matches
    good_matches = len(matches)

    # Calculate confidence (adjust as needed)
    confidence = (good_matches / len(kp1) if len(kp1) > 0 else 0) * 100

    return confidence

def compare_signatures_template_matching(image1_path, image2_path):
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

    img1 = cv2.resize(img1, (256, 256))
    img2 = cv2.resize(img2, (256, 256))

    res = cv2.matchTemplate(img2, img1, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    confidence = max_val*100


    return confidence


# Example usage
image1_path = 'original_signature.png'  # Replace with the actual path to your signature images
image2_path = 'test_signature.png'

ssim_confidence = compare_signatures_ssim(image1_path, image2_path)
orb_confidence = compare_signatures_orb(image1_path, image2_path)
template_confidence = compare_signatures_template_matching(image1_path, image2_path)


print(f"SSIM Confidence: {ssim_confidence:.2f}%")
print(f"ORB Confidence: {orb_confidence:.2f}%")
print(f"Template Matching Confidence: {template_confidence:.2f}%")

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
